In [1]:
import os
import pandas as pd
import numpy as np
import time
import pickle
import random
random.seed(0)

import gc

In [2]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'

In [4]:
MODEL_DATE = '20190819'

In [5]:
# input files
trained_models =  OUTPUT + MODEL_DATE + '_' + 'champs_models_nn_{}.h5'

In [6]:
# output files
merged_models =  OUTPUT + MODEL_DATE + '_' + 'merged_nn_{}.h5'

In [7]:
from keras.layers import Input
from keras.layers import Dense
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.layers import Concatenate

Using TensorFlow backend.


In [8]:
bond_types = ['1JHC', '1JHN', '2JHH', '2JHC', '2JHN', '3JHH', '3JHC', '3JHN'] #mapping order 3 type each

In [9]:
for b_target in bond_types:
    model_target = load_model(trained_models.format(b_target), compile=False)
    for l in model_target.layers:
        l.name = l.name + '_' + b_target

    models_add = []
    for b in bond_types:
        for i in range(3):
            model_add = load_model(trained_models.format(b), compile=False)
            for l in model_add.layers:
                l.name = l.name + '_' + b + '_' + 'x' + '_' + str(i)
            models_add.append(model_add)

    for b in bond_types:
        for i in range(3):
            model_add = load_model(trained_models.format(b), compile=False)
            for l in model_add.layers:
                l.name = l.name + '_' + b + '_' + 'y' + '_' + str(i)
            models_add.append(model_add)

    input_list = [model_target.input]
    in_list_temp = [models_add[i].input for i in range(len(models_add))]
    input_list.extend(in_list_temp)

    output_list = [model_target.output]
    out_list_temp = [models_add[i].output for i in range(len(models_add))]
    output_list.extend(out_list_temp)

    conc_output = Concatenate(name='concat_out')(output_list)
    dense_merge0 = Dense(10, name='dense_merge0')(conc_output)
    norm_merge0 = BatchNormalization(name='norm_merge0')(dense_merge0)
    dense_merge1 = Dense(1, name='dense_merge1')(norm_merge0)

    model_merge = Model(input_list, dense_merge1)
    
    model_merge.save(merged_models.format(b_target), include_optimizer=False)
    del model_merge